In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#Get the data
df_train = pd.read_csv('../input/torontohousingprices/dataset_final.csv')
df_train.columns

### Analyze 'Sold_price' 

In [ ]:
pd.set_option('display.float_format', lambda x: '%.f' % x)

#Descriptive statistics summary
df_train.describe()


In [ ]:
#Sold Price histogram
sns.distplot(df_train['Sold_price'])

### 'Sold_price' Relationship with Numerical Variables

In [ ]:
#Scatter plot 'Squarefootage'/'Sold_price'
var = 'Squarefootage'
data = pd.concat([df_train['Sold_price'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='Sold_price')

### 'Sold_price' Relationship with Location Variables

In [ ]:
print('There are', df_train['Community'].nunique(), 'communities in',
       df_train['Municipality District'].nunique(), 'districts.')

In [ ]:
# Get average 'Sold_price' across communities (neighbourhoods)
communities = df_train[['Sold_price','Community']].groupby(['Community'])

community_prices = pd.concat([communities.mean(), communities.count()], axis=1)
community_prices.columns = ['Avg. Price', 'Count']
community_prices_sorted = community_prices.sort_values(by=['Avg. Price'], ascending=False)
community_prices_sorted

In [ ]:
community_prices_sorted.plot.bar(figsize=(24,6))
community_prices_sorted['Count'].plot(kind='bar', color='Orange', secondary_y=True)

In [ ]:
# Get average 'Sold_price' across districts
districts = df_train[['Sold_price','Municipality District']].groupby(['Municipality District'])

districts_prices = pd.concat([districts.mean(), districts.count()], axis=1)
districts_prices.columns = ['Avg. Price', 'Count']
districts_prices_sorted = districts_prices.sort_values(by=['Avg. Price'], ascending=False)
districts_prices_sorted

In [ ]:
districts_prices_sorted.plot.bar(figsize=(24,6))
districts_prices_sorted['Count'].plot(kind='bar', color='Orange', secondary_y=True)

In [ ]:
# Communities with <5 data points
community_prices_sorted[community_prices_sorted['Count'] <= 15].sort_values(by=['Count'], ascending=True).to_csv("./output")

In [ ]:
#get map
neighbourhood_map = gpd.read_file('../input/folder/forAnalysis/Neighbourhoods/Neighbourhoods.shp')
neighbourhood_map['neighbourhood'] = neighbourhood_map['FIELD_7'].str.replace(' \(.+\)', '').str.lower()

#merge data
community_prices['Neighbourhood'] = community_prices.index.str.lower()
community_prices.reset_index(drop=True, inplace=True)

merged = neighbourhood_map.set_index('neighbourhood').join(community_prices.set_index('Neighbourhood'))
merged = merged.reset_index()
merged = merged.fillna(0)

#create heat map
fig, ax = plt.subplots(1, figsize=(40, 20))
ax.axis('off')
ax.set_title('Heat Map of House Prices by Neighbourhood in Toronto', fontdict={'fontsize': '40', 'fontweight' : '3'})

#create colorbar as a legend
color = 'Oranges'
vmin, vmax = 0, merged['Avg. Price'].max()
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
cbar.ax.tick_params(labelsize=15)

# plot map - annotated neighbourhoods with >5 count
merged.plot('Avg. Price', cmap=color, linewidth=0.8, ax=ax, edgecolor='0.8', figsize=(40, 20))
for idx, row in merged.iterrows():
    if(row['Count'] <= 5):
        plt.annotate(s=int(row['Count']), xy=(row['FIELD_11'], row['FIELD_12']),
                 horizontalalignment='center', fontsize='large', color='black', wrap=True)
plt.show()

In [ ]:
# List of 0 count neighbourhoods
merged[merged['Count']<16][['neighbourhood','Avg. Price', 'Count']].sort_values(by=['Count'], ascending=True).to_csv('./output.csv')

### Correlation matrix

In [ ]:
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

## Begin Data Engineering

### Type

In [ ]:
# Get average 'Sold_price' across communities (neighbourhoods)
types = df_train[['Sold_price','Type']].groupby(['Type'])

type_prices = pd.concat([types.mean(), types.count()], axis=1)
type_prices.columns = ['Avg. Price', 'Count']
type_prices.sort_values(by=['Avg. Price'], ascending=False)

#### Use one hot encoding for Type data

In [ ]:
oneHot = OneHotEncoder(handle_unknown='ignore')
ce_ohe = ce.OneHotEncoder(cols = ['Type'], use_cat_names=True)
df_train= ce_ohe.fit_transform(df_train)
print(ce_ohe)
df_train.head()

### Style

In [ ]:
# Get average 'Sold_price' across communities (neighbourhoods)
styles = df_train[['Sold_price','Style']].groupby(['Style'])

style_prices = pd.concat([styles.mean(), styles.count()], axis=1)
style_prices.columns = ['Avg. Price', 'Count']
style_prices.sort_values(by=['Avg. Price'], ascending=False)

In [ ]:
oneHot = OneHotEncoder(handle_unknown='ignore')
ce_ohe = ce.OneHotEncoder(cols = ['Style'], use_cat_names=True)
df_train= ce_ohe.fit_transform(df_train)
print(ce_ohe)
df_train.head()

### District and Community Encoding

In [ ]:
# Bin districts
df_train['district_bin'] = df_train['Municipality District']
df_train['district_bin'][df_train['Municipality District'] == 'C12'] = 'district_bin_0'
df_train['district_bin'][df_train['Municipality District'] == 'C04'] = 'district_bin_1'
df_train['district_bin'][df_train['Municipality District'] == 'C09'] = 'district_bin_2'
df_train['district_bin'][df_train['Municipality District'] == 'C03'] = 'district_bin_3'
df_train['district_bin'][df_train['Municipality District'] == 'E02'] = 'district_bin_4'
df_train['district_bin'][df_train['Municipality District'] == 'C02'] = 'district_bin_4'
df_train['district_bin'][df_train['Municipality District'] == 'W07'] = 'district_bin_5'
df_train['district_bin'][df_train['Municipality District'] == 'W02'] = 'district_bin_6'
df_train['district_bin'][df_train['Municipality District'] == 'E01'] = 'district_bin_6'
df_train['district_bin'][df_train['Municipality District'] == 'W01'] = 'district_bin_6'
df_train['district_bin'][df_train['Municipality District'] == 'C13'] = 'district_bin_7'
df_train['district_bin'][df_train['Municipality District'] == 'E03'] = 'district_bin_7'
df_train['district_bin'][df_train['Municipality District'] == 'C06'] = 'district_bin_7'
df_train['district_bin'][df_train['Municipality District'] == 'C07'] = 'district_bin_8'
df_train['district_bin'][df_train['Municipality District'] == 'E06'] = 'district_bin_8'
df_train['district_bin'][df_train['Municipality District'] == 'W03'] = 'district_bin_8'
df_train['district_bin'][df_train['Municipality District'] == 'C14'] = 'district_bin_8'
df_train['district_bin'][df_train['Municipality District'] == 'W08'] = 'district_bin_8'
df_train['district_bin'][df_train['Municipality District'] == 'C11'] = 'district_bin_9'
df_train['district_bin'][df_train['Municipality District'] == 'E10'] = 'district_bin_9'
df_train['district_bin'][df_train['Municipality District'] == 'C10'] = 'district_bin_9'
df_train['district_bin'][df_train['Municipality District'] == 'E08'] = 'district_bin_10'
df_train['district_bin'][df_train['Municipality District'] == 'W06'] = 'district_bin_10'
df_train['district_bin'][df_train['Municipality District'] == 'C01'] = 'district_bin_10'
df_train['district_bin'][df_train['Municipality District'] == 'C08'] = 'district_bin_11'
df_train['district_bin'][df_train['Municipality District'] == 'W09'] = 'district_bin_11'
df_train['district_bin'][df_train['Municipality District'] == 'W04'] = 'district_bin_11'
df_train['district_bin'][df_train['Municipality District'] == 'E07'] = 'district_bin_12'
df_train['district_bin'][df_train['Municipality District'] == 'W05'] = 'district_bin_12'
df_train['district_bin'][df_train['Municipality District'] == 'E04'] = 'district_bin_12'
df_train['district_bin'][df_train['Municipality District'] == 'C15'] = 'district_bin_12'
df_train['district_bin'][df_train['Municipality District'] == 'E05'] = 'district_bin_12'
df_train['district_bin'][df_train['Municipality District'] == 'E11'] = 'district_bin_13'
df_train['district_bin'][df_train['Municipality District'] == 'E09'] = 'district_bin_13'
df_train['district_bin'][df_train['Municipality District'] == 'W10'] = 'district_bin_14'

# One-hot encode districts
oneHot = OneHotEncoder(handle_unknown='ignore')
ce_ohe = ce.OneHotEncoder(cols = ['district_bin'], use_cat_names=True)
df_train= ce_ohe.fit_transform(df_train)
df_train.head()

In [ ]:
def bin_range(start_of_range,start_of_next_range):
    return range(list(community_prices_sorted.index.values).index(start_of_range),list(community_prices_sorted.index.values).index(start_of_next_range))

# Set Community Bins
bins = [bin_range('Bridle Path-Sunnybrook-York Mills','Lawrence Park South')],\
        [bin_range('Lawrence Park South','Lawrence Park North')],\
        [bin_range('Lawrence Park North','Princess-Rosethorn')],\
        [bin_range('Princess-Rosethorn','Bedford Park-Nortown')],\
        [bin_range('Bedford Park-Nortown','St. Andrew-Windfields')],\
        [bin_range('St. Andrew-Windfields','Lambton Baby Point')],\
        [bin_range('Lambton Baby Point','Danforth')],\
        [bin_range('Danforth','Highland Creek')],\
        [bin_range('Highland Creek','Danforth Village-East York')],\
        [bin_range('Danforth Village-East York','Edenbridge-Humber Valley')],\
        [bin_range('Edenbridge-Humber Valley','Woodbine Corridor')],\
        [bin_range('Woodbine Corridor','Cabbagetown-South St. James Town')],\
        [bin_range('Cabbagetown-South St. James Town','University')],\
        [bin_range('University','Dovercourt-Wallace Emerson-Junction')],\
        [bin_range('Dovercourt-Wallace Emerson-Junction','Rouge E10')],\
        [bin_range('Rouge E10','Waterfront Communities C8')],\
        [bin_range('Waterfront Communities C8','Rockcliffe-Smythe')],\
        [bin_range('Rockcliffe-Smythe','Briar Hill-Belgravia')],\
        [bin_range('Briar Hill-Belgravia','Bayview Village')],\
        [bin_range('Bayview Village','Dorset Park')],\
        [bin_range('Dorset Park','Eglinton East')],\
        [bin_range('Eglinton East','Mount Olive-Silverstone-Jamestown')],\
        [bin_range('Mount Olive-Silverstone-Jamestown','Elms-Old Rexdale')],\
        [bin_range('Elms-Old Rexdale','Black Creek')]

df_train['community_bin'] = np.nan

for idx,item in enumerate(bins):
    bin_name = 'community_bin_' + str(idx)
    df_train['community_bin'][df_train['Community'].isin(community_prices_sorted.index.values[item])] = bin_name
    last_bin_name = bin_name

# Set end of last range 
df_train['community_bin'][df_train['Community'] == 'Black Creek'] = last_bin_name
    
# One-hot encode Community Bins
oneHot = OneHotEncoder(handle_unknown='ignore')
ce_ohe = ce.OneHotEncoder(cols = ['community_bin'], use_cat_names=True)
df_train= ce_ohe.fit_transform(df_train)
df_train.head()

### Combine Bedrooms and Den Columns

In [ ]:
beds = df_train[['Bedrooms','Dens']]
multiplier = 0.48

beds_prices = pd.concat([beds.Bedrooms, beds.Dens, beds.Bedrooms.astype(float) + multiplier * beds.Dens.astype(float)], axis=1)
pd.options.display.float_format = '{:,.2f}'.format
beds_prices.columns = ['Bedrooms', 'Dens', 'Combined']

beds_prices

#### Compare Combined Correlation

In [ ]:
#Combined vs. Bedrooms vs. Dens 
bedrooms = pd.concat([df_train['Sold_price'], beds_prices['Bedrooms']], axis=1)
bedrooms.plot.scatter(x='Bedrooms', y='Sold_price')

dens = pd.concat([df_train['Sold_price'], beds_prices['Dens']], axis=1)
dens.plot.scatter(x='Dens', y='Sold_price')

combined = pd.concat([df_train['Sold_price'], beds_prices['Combined']], axis=1)
combined.plot.scatter(x='Combined', y='Sold_price')

print('Correlation:\nBedrooms - ', df_train['Sold_price'].corr(beds_prices['Bedrooms']),
                  '\nDens     - ', df_train['Sold_price'].corr(beds_prices['Dens']),
                  '\nCombined - ', df_train['Sold_price'].corr(beds_prices['Combined']))


#### Optimize Dens Multiplier

In [ ]:
# Correlation score of 'Combined' as a function of the multiplier
def combined_correlation(multiplier):
    return df_train['Sold_price'].corr(beds.Bedrooms + beds.Dens * multiplier)

# Create table with possible multiplier values on each row
optimize = pd.DataFrame(columns=['multiplier','combined_correlation'])
optimize['multiplier'] = np.linspace(0,1,1000) # 1000 values b/w 0 and 1

# Calculate Correlation score for each row
for idx, row in optimize.iterrows():   
    optimize['combined_correlation'][idx] = combined_correlation(optimize['multiplier'][idx])

# Get multiplier for maximum Correlation score
print('multiplier: ', optimize.loc[optimize['combined_correlation']==optimize['combined_correlation'].max(), 'multiplier'].iloc[0])


### Bathrooms, Kitchens, Rooms, Parking Total Correlations

In [ ]:
bathrooms = pd.concat([df_train['Sold_price'], df_train['Bathrooms']], axis=1)
bathrooms.plot.scatter(x='Bathrooms', y='Sold_price')

kitchen = pd.concat([df_train['Sold_price'], df_train['Kitchens']], axis=1)
kitchen.plot.scatter(x='Kitchens', y='Sold_price')

rooms = pd.concat([df_train['Sold_price'], df_train['Rooms']], axis=1)
rooms.plot.scatter(x='Rooms', y='Sold_price')

parking = pd.concat([df_train['Sold_price'], df_train['Parking Total']], axis=1)
parking.plot.scatter(x='Parking Total', y='Sold_price')

print('Correlation:\nBathrooms - ', df_train['Sold_price'].corr(df_train['Bathrooms']),
                  '\nKitchens  - ', df_train['Sold_price'].corr(df_train['Kitchens']),
                  '\nRooms     - ', df_train['Sold_price'].corr(df_train['Dens']),
                  '\nParking   - ', df_train['Sold_price'].corr(df_train['Parking Total']))

### Transform Skewed Data

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['Squarefootage'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['Squarefootage'], plot=plt)

In [ ]:
# log squarefootage
df_train['log(Squarefootage)'] = np.log(df_train['Squarefootage'])

sns.distplot(df_train['log(Squarefootage)'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['log(Squarefootage)'], plot=plt)

### Drop latitude, longitude, kitchens, squarefootage, rooms rows

In [ ]:
df_train = df_train.drop(['longitude', 'latitude', 'Rooms', 'Kitchens', 'Squarefootage', 'Community', 'Municipality District'], axis=1)
df_train.head()

### Drop/Transform Outliers

In [ ]:
# Drop 7 dens
df_train = df_train.drop(df_train[df_train['Dens']==7].index.values)

# Drop Bathrooms 9+
df_train = df_train.drop(df_train[df_train['Bathrooms']>=9].index.values)
df_train = df_train.drop(df_train[df_train['Bathrooms']==0].index.values)

# Drop Parking 9+
df_train = df_train.drop(df_train[df_train['Parking Total']>=9].index.values)

# Round down .5 Parking Total
df_train["Parking Total"].replace({"1.5": "1", "3.5": "3", "5.5": "5"}, inplace=True)

### Normalize values to help NN train

In [ ]:
# Scale Data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(df_train)

scaled_train = pd.DataFrame(data=scaled_train, columns=df_train.columns)

In [ ]:
print("Columns")
print(scaled_train.columns)
# Get scaled values
# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: sold_price values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[0], scaler.min_[0]))
multiplied_by = scaler.scale_[0]
added = scaler.min_[0]
print("Multiplied by")
print(scaler.scale_)
print("Added")
print(scaler.min_)

### Separate Dataset into Train (70%), Validation (20%),Test (10%)

In [ ]:
# Split Data
train, test = np.split(scaled_train.sample(frac=1, random_state=42),\
                        [int(.9*len(scaled_train))])

sns.distplot(train['Sold_price'])
sns.distplot(test['Sold_price'])

print(len(train))
print(len(test))
print(len(test)+len(train))

### Setup input, label data

In [ ]:
y_train_data = train.values[:,0]
y_test_data = test.values[:,0]
x_train_data = train.values[:,1:]
x_test_data = test.values[:,1:]

In [ ]:
# Get input shape
len(x_train_data[0])

## Finished Data Engineering
## Begin Neural Network

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [ ]:
# Build model
model = Sequential()

model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Train the model
hist = model.fit(
    x_train_data,
    y_train_data,
    epochs=50,
    shuffle=True,
    verbose=2,
    validation_data=(x_test_data, y_test_data)
)

### Visualize Results

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

In [ ]:
model.save('model')

In [ ]:
loaded_model = keras.models.load_model('model')

In [ ]:
# Inference
prediction = loaded_model.predict(x_test_data[:1])
y_0 = prediction[0][0]
print('Prediction with scaling - {}',format(y_0))
y_0 -= added
y_0 /= multiplied_by
print("Housing Price Prediction  - ${}".format(y_0))
actual = y_test_data[:1]
print('Actual with scaling - {}',format(actual))
actual -= added
actual /= multiplied_by
print("Housing Price Prediction  - ${}".format(actual))